# Test CodeAutoMergeEditBlock

This notebook tests the CodeAutoMergeEditBlock class's ability to parse different code block formats.

In [1]:
from autocoder.common.code_auto_merge_editblock import CodeAutoMergeEditBlock
from autocoder.common import AutoCoderArgs
import byzerllm

In [2]:
# Test data with two different formats
test_input = """
```python
##File: src/autocoder/index/index.py
<<<<<<< SEARCH
    # Print final statistics
    logger.info("\n=== Build Index and Filter Files Summary ===\n")
    return source_code
=======
    # Print final statistics
    logger.info("\n=== Build Index and Filter Files Summary ===\n")
    # Send filter end event
    if args.request_id:
        queue_communicate.send_event_no_wait(
            request_id=args.request_id,
            event=CommunicateEvent(
                event_type=CommunicateEventType.CODE_INDEX_FILTER_END.value,
                data=f"Completed filtering. Final files selected: {stats['final_files']}"
            )
        )
    return source_code
>>>>>>> REPLACE
```

```python:src/autocoder/index/index.py
<<<<<<< SEARCH
    # Print final statistics
    logger.info("\n=== Build Index and Filter Files Summary ===\n")
    return source_code
=======
    # Print final statistics
    logger.info("\n=== Build Index and Filter Files Summary ===\n")
    # Send filter end event
    if args.request_id:
        queue_communicate.send_event_no_wait(
            request_id=args.request_id,
            event=CommunicateEvent(
                event_type=CommunicateEventType.CODE_INDEX_FILTER_END.value,
                data=f"Completed filtering. Final files selected: {stats['final_files']}"
            )
        )
    return source_code
>>>>>>> REPLACE
```
"""

In [3]:
# Initialize CodeAutoMergeEditBlock
llm = byzerllm.ByzerLLM.from_default_model("deepseek_chat")
args = AutoCoderArgs()
merger = CodeAutoMergeEditBlock(llm, args)

# Parse the test input
results = merger.parse_whole_text(test_input)

# Verify results
assert len(results) == 2, f"Expected 2 results, got {len(results)}"

# Both results should have the same path
assert results[0].path == "src/autocoder/index/index.py"
assert results[1].path == "src/autocoder/index/index.py"

# Both results should have the same content structure
for result in results:
    assert "<<<<<<< SEARCH" in result.content
    assert "=======" in result.content
    assert ">>>>>>> REPLACE" in result.content
    
print("All tests passed!")

2024-11-30 21:39:36.770 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-11-30 21:39:36,828	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-11-30 21:39:36,857	INFO worker.py:1740 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


All tests passed!


In [4]:
# Let's also test the get_edits method to ensure we can extract the search and replace blocks correctly
edits = merger.get_edits(test_input)

# We should have 2 edits
assert len(edits) == 2, f"Expected 2 edits, got {len(edits)}"

# Each edit should be a tuple of (path, head, update)
for path, head, update in edits:
    assert path == "src/autocoder/index/index.py"
    assert "# Print final statistics" in head
    assert "# Send filter end event" in update
    
print("Edit extraction tests passed!")

Edit extraction tests passed!
